In [1]:
import pymongo
import pprint
import glob
import pandas as pd

db_connect = pymongo.MongoClient("localhost", 27017)
database_name = 'ye'
db = db_connect[database_name]
collection = db.list_collection_names(include_system_collections=False)

In [2]:
gr = db.genome_records

# Genbank example

In [3]:
g = gr.find_one({"name": "NZ_CP010029.1"})
accession_id = g['refseq_accession_id']

In [4]:
g.keys()

dict_keys(['_id', 'name', 'species', 'description', 'sequence', 'assembly_name', 'biosample', 'bioproject', 'date', 'wgs_project', 'genome_coverage', 'organism', 'taxid', 'assembly_type', 'release_type', 'assembly_level', 'genome_representation', 'expected_final_version', 'excluded', 'genbank_accession_id', 'refseq_accession_id', 'r_g_identical', 'plasmid', 'present', 'hits', 'references', 'tags'])

In [5]:
print (g['name'])
print (g['species'])
print (accession_id)

NZ_CP010029.1
Yersinia entomophaga
GCF_001656035.1


# Load in the feature table

In [6]:
filepath = "/Users/gabefoley/Dropbox/PhD/Projects/Phylo_Island/2020/20201119_Testing_Feature_Tables/refseq/bacteria/"
feature_path = glob.glob(filepath + accession_id + "/*_feature_table.txt.gz")[0]
df = pd.read_csv(feature_path, sep='\t', compression='gzip')
df.rename(columns = {'# feature':'feature'}, inplace = True)
df.head(5)

,feature,class,assembly,assembly_unit,seq_type,chromosome,genomic_accession,start,end,strand,product_accession,non-redundant_refseq,related_accession,name,symbol,GeneID,locus_tag,feature_interval_length,product_length,attributes
0,gene,pseudogene,GCF_001656035.1,Primary Assembly,chromosome,NaN,NZ_CP010029.1,24,1091,-,NaN,NaN,NaN,NaN,NaN,NaN,PL78_RS20065,1068,NaN,partial;pseudo;old_locus_tag=PL78_00005
1,CDS,without_protein,GCF_001656035.1,Primary Assembly,chromosome,NaN,NZ_CP010029.1,24,1091,-,NaN,NaN,NaN,hypothetical protein,NaN,NaN,PL78_RS20065,1068,NaN,partial;pseudo
2,gene,protein_coding,GCF_001656035.1,Primary Assembly,chromosome,NaN,NZ_CP010029.1,1352,1534,-,NaN,NaN,NaN,NaN,NaN,NaN,PL78_RS00010,183,NaN,old_locus_tag=PL78_00010
3,CDS,with_protein,GCF_001656035.1,Primary Assembly,chromosome,NaN,NZ_CP010029.1,1352,1534,-,WP_064512099.1,WP_064512099.1,NaN,hypothetical protein,NaN,NaN,PL78_RS00010,183,60.0,NaN
4,gene,protein_coding,GCF_001656035.1,Primary Assembly,chromosome,NaN,NZ_CP010029.1,1700,1978,+,NaN,NaN,NaN,NaN,NaN,NaN,PL78_RS00015,279,NaN,old_locus_tag=PL78_00015


In [8]:
for hit in g['hits']:
    if 'expanded' in hit['region']:
        print (hit['region'])
        
        genome_interval = pd.Interval(int(hit['start']), int(hit['end']), closed='both')
        
        print (genome_interval)
                    
        filtered = df.loc[df['assembly'] == g['refseq_accession_id']]
        
        filtered.head(5)

        for gene in filtered.itertuples():
            if gene.feature == 'CDS':
                if not (gene.start > int(hit['end'])) and not (gene.end < int(hit['start'])):
                    check_interval = pd.Interval(gene.start, gene.end, closed='both')

                    if check_interval.overlaps(genome_interval):
                        print ('Found one')
                        print(gene.name, gene.product_accession, gene.start, gene.end)
                    #
                    # hit = models.Hits(object_id, record.name, new_reg, new_score, new_start, new_end,
                    #                   expand,
                    #                   strand, sequence)
                    #
                    # print('We are adding a refseq sequence in')
                    #
                    # print(hit.start)
                    # print(hit.end)
                    # curr.hits.append(hit)
                    # curr.save()



A1_expanded
[798387, 801882]
Found one
toxin WP_064513225.1 798388 801882
Found one
hypothetical protein WP_064513227.1 801875 805969
A2_expanded
[801874, 805969]
Found one
toxin WP_064513225.1 798388 801882
Found one
hypothetical protein WP_064513227.1 801875 805969
TcB_expanded
[808162, 812626]
Found one
virulence protein WP_064513231.1 808163 812626
TcC_expanded
[815831, 818780]
Found one
hypothetical protein WP_084414279.1 815868 818780
TcC_expanded
[4177811, 4180709]
Found one
RHS repeat protein nan 4178646 4180709
TcC_expanded
[812721, 815646]
Found one
hypothetical protein WP_064513233.1 812722 815646
TcdA1_expanded
[801874, 805969]
Found one
toxin WP_064513225.1 798388 801882
Found one
hypothetical protein WP_064513227.1 801875 805969
Chitinase_expanded
[796661, 798302]
Found one
chitinase WP_064513223.1 796662 798302
Chitinase_expanded
[806066, 807968]
Found one
chitinase WP_064513229.1 806067 807968
region1_expanded
[801877, 802753]
Found one
toxin WP_064513225.1 798388 80188

In [136]:
filtered.head(5)

,feature,class,assembly,assembly_unit,seq_type,chromosome,genomic_accession,start,end,strand,product_accession,non-redundant_refseq,related_accession,name,symbol,GeneID,locus_tag,feature_interval_length,product_length,attributes
